**Abstract.**

# Regression

## Modules

In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot

# References

[1] https://machinelearningmastery.com/calculate-feature-importance-with-python/

## Tutorials

[1] [How to Calculate Feature Importance With Python](https://machinelearningmastery.com/calculate-feature-importance-with-python/)

[2] [How to Choose a Feature Selection Method For Machine Learning](https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/)

[3] [How to Perform Feature Selection with Categorical Data](https://machinelearningmastery.com/feature-selection-with-categorical-data/)

[4] [Feature Importance and Feature Selection With XGBoost in Python](https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/)

[5] [Feature Selection For Machine Learning in Python](https://machinelearningmastery.com/feature-selection-machine-learning-python/)

[6] A[n Introduction to Feature Selection](https://machinelearningmastery.com/an-introduction-to-feature-selection/)

## Books

Kuhn, M. and Johnson, K., "Applied Predictive Modeling", Springer, 1st ed., 2018

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w